In [ ]:
%pylab inline
from tmqr.settings import *
from tmqrfeed import DataManager
from pymongo import MongoClient
from tmqrindex.index_base import IndexBase
from datetime import datetime
import re
import pandas as pd


import seaborn as sns
#import cufflinks as cf
#cf.go_offline()

import pandas as pd
import numpy as np
import scipy

#import matplotlib
#%matplotlib notebook

figsize(15,7)

In [ ]:
from sklearn import (pipeline, preprocessing, ensemble, neighbors, linear_model, neural_network, cluster, metrics, decomposition,
                     naive_bayes, calibration, svm, multioutput,
                     feature_selection, discriminant_analysis, model_selection, multiclass
                    )

In [ ]:
client = MongoClient(MONGO_CONNSTR)
db = client[MONGO_DB]

dm = DataManager()

In [ ]:
def get_exo_list(exo_filter='*', return_names=True):
    """
    Return EXO list stored in MongoDB V2
    :param exo_filter: '*' - include all, wildcard is allowed (like, 'ES_Bullish*')
    :param return_names: if True returns names list of EXO, otherwize returns MongoDB data collection list
    :return: list of EXO names
    """
    re_val = exo_filter.replace('*','.*')

    data = db['index_data'].find({'name': re.compile(re_val, re.IGNORECASE)})
    if return_names:
        return [exo['name'] for exo in data]
    else:
        return list(data)

In [ ]:
def universal_fisher_transform(series, transform_with='arctanh'):
    '"transform_with" options - "tanh", "arctanh"'
    
    # Centering the series
    series = series.expanding().apply(lambda x: preprocessing.StandardScaler().fit_transform(x.reshape(-1, 1)
                                                                                                          ).ravel()[-1])

    # limiting it to -0.999 > x < 0.999
    series = series.expanding().apply(lambda x: preprocessing.MinMaxScaler(feature_range=(-0.999,0.999)
                                                                                        ).fit_transform(x.reshape(-1, 1)).ravel()[-1])

    if transform_with == 'arctanh':
        ft_ser = np.arctanh(series)
        
    elif transform_with == 'tanh':
        ft_ser = np.tanh(series)
        
    return ft_ser

In [ ]:
#exo_filter = 'neutralOnly'     # All 
exo_filter = '*'  # ES only
#exo_filter = '*'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

exo_dict = {}
for exo in get_exo_list(exo_filter, return_names=False):
    idx = IndexBase.deserialize(dm, exo, as_readonly=True)
    exo_dict[idx.index_name] = idx

In [ ]:
for exo_name, exo in exo_dict.items():
    exo_df = exo.data
    
    if len(exo_df) < 200:
        print("{0:<70} [NODATA DataLen: {1}]".format(exo.index_name, len(exo_df)))
    elif (datetime.now().date() - exo_df.index.date[-1]).days > 4:
        print("{0:<70} [DELAYED: LastDate: {1}]".format(exo.index_name, exo_df.index[-1]))
    else:
        print("{0:<70} [OK]".format(exo.index_name))

In [ ]:
exo_df = exo_dict['US.HO_ContFutEOD'].data.copy()

#exo_df['c'] = ((exo_df.c - exo_df.c.ewm(100).mean()) / exo_df.c.ewm(100).std())

In [ ]:
pos_move = exo_df.c > exo_df.c.shift(1)
exo_df.loc[pos_move == True, 'v'] = 1

In [ ]:
exo_df.tail()

In [ ]:
vwap = ((exo_df.c * exo_df.v).cumsum() / (exo_df.v).cumsum())

In [ ]:
exo_df.c.plot()
vwap.plot(label='VWAP', legend=True)
exo_df.c.expanding().mean().plot(label='Expanding mean', legend=True);

In [ ]:
temp_df = pd.DataFrame(index=exo_df.index)

for y in np.unique(exo_df.index.year):
    exo_df_slice = exo_df[exo_df.index.year == y]
    vwap_y = ((exo_df_slice.c * exo_df_slice.v).cumsum() / exo_df_slice.v.cumsum())

    temp_df['vwap_yearly_{}'.format(y)] = vwap_y
    
    for q in np.unique(exo_df.index.quarter):
        exo_df_slice = exo_df[(exo_df.index.year == y) & (exo_df.index.quarter == q)]
        vwap = ((exo_df_slice.c * exo_df_slice.v).cumsum() / exo_df_slice.v.cumsum())

        temp_df['vwap_qtr_{}_{}'.format(q, y)] = vwap
        
    for w in np.unique(exo_df.index.weekofyear):
        exo_df_slice = exo_df[(exo_df.index.year == y) & (exo_df.index.weekofyear == w)]
        vwap = ((exo_df_slice.c * exo_df_slice.v).cumsum() / exo_df_slice.v.cumsum())

        temp_df['vwap_weekly_{}_{}'.format(w, y)] = vwap
        
    for m in np.unique(exo_df.index.month):
        exo_df_slice = exo_df[(exo_df.index.year == y) & (exo_df.index.month == m)]
        vwap = ((exo_df_slice.c * exo_df_slice.v).cumsum() / exo_df_slice.v.cumsum())

        temp_df['vwap_monthly_{}_{}'.format(m, y)] = vwap
        
exo_df['vwap_weekly'] = temp_df.filter(regex='vwap_weekly').sum(axis=1)
exo_df['vwap_monthly'] = temp_df.filter(regex='vwap_month').sum(axis=1)
exo_df['vwap_quarterly'] = temp_df.filter(regex='vwap_qtr').sum(axis=1)
exo_df['vwap_yearly'] = temp_df.filter(regex='vwap_year').sum(axis=1)

del temp_df

In [ ]:
l = list(str('vhlo'))
l.append('exec')

#exo_df.drop(list(str('vhloc')), axis=1).iloc[700:1000:].plot();
exo_df.drop(l, axis=1).iloc[-700:].plot(cmap='Vega10');